## player quality

The purpose of this notebook is to determine the quality of players per team and position for the whole season. Each player will be manually assigned a value of 1 or 2 based on their productivity. This will show the quantity of elite players each team have and assist with estimating the value of each roster position on team success. 

###  import modules

In [55]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import data 

In [56]:
dm = pd.read_csv('play_by_play.csv')
dm = dm.drop('Unnamed: 0', axis=1)

/home/kmongeon/anaconda3/envs/datascience1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
dm.head(10)

,Season,GameNumber,GameDate,Period,AdvantageType,Zone,EventNumber,EventType,EventDetail,EventTeamCode,EventPlayerNumber,EventPlayerName,EventTimeFromZero,EventTimeFromTwenty,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition,ShotType,ShotResult,Length,PenaltyType
0,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,11.0,C,TOR,37.0,C,NaN,NaN,NaN,NaN
1,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,21.0,R,TOR,9.0,R,NaN,NaN,NaN,NaN
2,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,57.0,L,TOR,11.0,L,NaN,NaN,NaN,NaN
3,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,26.0,D,TOR,3.0,D,NaN,NaN,NaN,NaN
4,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,75.0,D,TOR,22.0,D,NaN,NaN,NaN,NaN
5,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,31.0,G,TOR,35.0,G,NaN,NaN,NaN,NaN
6,2010,20001,2010-10-07,1,EV,O,3,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",TOR,37.0,BRENT,15,1185,MTL,11.0,C,TOR,37.0,C,NaN,NaN,NaN,NaN
7,2010,20001,2010-10-07,1,EV,O,3,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",TOR,37.0,BRENT,15,1185,MTL,21.0,R,TOR,9.0,R,NaN,NaN,NaN,NaN
8,2010,20001,2010-10-07,1,EV,O,3,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",TOR,37.0,BRENT,15,1185,MTL,57.0,L,TOR,11.0,L,NaN,NaN,NaN,NaN
9,2010,20001,2010-10-07,1,EV,O,3,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",TOR,37.0,BRENT,15,1185,MTL,26.0,D,TOR,3.0,D,NaN,NaN,NaN,NaN


In [58]:
dm.shape

(1531666, 24)

create new data set and keep variables: 
- (a) game number.
- (b) visitor team information.
- (c) home team information.

In [59]:
df = dm[['Season', 'GameNumber', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition']]

In [60]:
df.shape

(1531666, 8)

- reshape the data to have home and visitor team observatons under the same coloumns. 

In [61]:
a = [col for col in df.columns if 'Player' in col]
b = [col for col in df.columns if 'Position' in col]
c = [col for col in df.columns if 'TeamCode' in col]
df = pd.lreshape(df, {'Player' : a, 'Position' : b, 'TeamCode' : c})

In [62]:
df.head(5)

,GameNumber,Season,Player,Position,TeamCode
0,20001,2010,11.0,C,MTL
1,20001,2010,21.0,R,MTL
2,20001,2010,57.0,L,MTL
3,20001,2010,26.0,D,MTL
4,20001,2010,75.0,D,MTL


In [63]:
df.shape

(3063332, 5)

- group data by game number, team and player to drop duplicates and keep only one observation per player per game.

In [64]:
df = df.drop_duplicates(['GameNumber', 'TeamCode', 'Player'])
df = df[['Season', 'GameNumber', 'TeamCode', 'Player', 'Position']]
df = df.rename(columns={'Player': 'PlayerNumber', 'Position': 'PlayerPosition' })
df = df.sort_values(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'PlayerNumber'], ascending=[True, True, True, True, True])

In [65]:
df.shape

(46920, 5)

- store player quality to csv file

In [66]:
df.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerPosition
0,2010,20001,MTL,11.0,C
12,2010,20001,MTL,14.0,C
54,2010,20001,MTL,15.0,C
78,2010,20001,MTL,17.0,C
55,2010,20001,MTL,40.0,C


In [67]:
#df.to_csv('out_data/player_quality.csv', index='False', sep=',')

In [68]:
df.shape

(46920, 5)

#### roster position follows.

- group data by team and player to keep one observation per player for the whole season (drop duplicates).

In [69]:
dc = df[['Season', 'TeamCode', 'PlayerNumber', 'PlayerPosition']]

In [70]:
dc = dc.drop_duplicates(['TeamCode', 'PlayerNumber', 'PlayerPosition'])
dc = dc.sort_values(['TeamCode', 'PlayerPosition', 'PlayerNumber'], ascending=[True, True, True])

In [71]:
dc.shape

(1059, 4)

- save the data set and manually rank players per team.

In [72]:
dc.head(5)

,Season,TeamCode,PlayerNumber,PlayerPosition
9607,2010,ANA,11.0,C
186052,2010,ANA,12.0,C
2264542,2010,ANA,14.0,C
9625,2010,ANA,15.0,C
1609648,2010,ANA,20.0,C


In [73]:
dc.to_csv('player_rank.csv', index='False', sep=',')

#### next step is to rank players manually and  merge player rank into play by play

- import manual player rank csv file.

In [74]:
dr = pd.read_csv('player_rank_manual.csv')
dr = dr.drop('Unnamed: 0', axis=1)

In [75]:
dr.shape

(1059, 5)

- **rename columns** for event team and player in order to reshape data. 

In [76]:
dr.head()

,Season,TeamCode,PlayerNumber,PlayerPosition,Rank
0,2010,ANA,11,C,2
1,2010,ANA,12,C,2
2,2010,ANA,14,C,2
3,2010,ANA,15,C,1
4,2010,ANA,20,C,2


In [77]:
dm = dm.rename(columns={'EventPlayerNumber': 'EventPNumber', 'EventTeamCode': 'EventTeam', 'EventPlayerName': 'EventPName' })

### reshape data for home and visitor  observations to be under the same columns.

In [78]:
a = [col for col in dm.columns if 'Player' in col]
b = [col for col in dm.columns if 'Position' in col]
c = [col for col in dm.columns if 'TeamCode' in col]
dm = pd.lreshape(dm, {'Player' : a, 'Position' : b, 'TeamCode' : c})

In [79]:
dm = dm.rename(columns={'Player': 'PlayerNumber', 'Position': 'PlayerPosition', 'EventPNumber': 'EventPlayerNumber', 'EventTeam': 'EventTeamCode', 'EventPName': 'EventPlayerName'})
dm = dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])

In [80]:
dm.shape

(3063332, 21)

### merge player rank data set onto play_by_play

In [44]:
dm = dm.merge(dr, on=['Season', 'TeamCode', 'PlayerNumber', 'PlayerPosition'], how='left')
dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [45]:
dm = dm[['Season', 'GameNumber', 'GameDate', 'Period', 'AdvantageType', 'Zone', 'EventNumber', 'EventType',  'EventDetail', 'EventTeamCode', 'EventPlayerNumber', 'EventPlayerName',  'EventTimeFromZero', 'EventTimeFromTwenty', 'TeamCode', 'PlayerNumber', 'PlayerPosition', 'ShotType', 'ShotResult', 'Length', 'PenaltyType', 'Rank']]
dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [46]:
dm.shape

(3063332, 22)

- check the data set to see if and which columns contain NaN values.

In [47]:
dm.isnull().sum()

Season                       0
GameNumber                   0
GameDate                     0
Period                       0
AdvantageType                0
Zone                         0
EventNumber                  0
EventType                    0
EventDetail                  0
EventTeamCode                0
EventPlayerNumber            0
EventPlayerName              0
EventTimeFromZero            0
EventTimeFromTwenty          0
TeamCode                     0
PlayerNumber                 0
PlayerPosition               0
ShotType               1749130
ShotResult             2789200
Length                 2084206
PenaltyType            2946770
Rank                         0
dtype: int64

In [48]:
dm.head()

,Season,GameNumber,GameDate,Period,AdvantageType,Zone,EventNumber,EventType,EventDetail,EventTeamCode,EventPlayerNumber,EventPlayerName,EventTimeFromZero,EventTimeFromTwenty,TeamCode,PlayerNumber,PlayerPosition,ShotType,ShotResult,Length,PenaltyType,Rank
0,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,11,C,NaN,NaN,NaN,NaN,2
1,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,21,R,NaN,NaN,NaN,NaN,2
2,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,57,L,NaN,NaN,NaN,NaN,2
3,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,26,D,NaN,NaN,NaN,NaN,2
4,2010,20001,2010-10-07,1,EV,N,1,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,MTL,11.0,GOMEZ,0,1200,MTL,75,D,NaN,NaN,NaN,NaN,2


- save new data set.

In [49]:
dm.to_csv('play_by_play_with_player_rank.csv', index='False', sep=',')

#### the next step is to estimate the value of each roster position.